<a href="https://colab.research.google.com/github/roamin9/llama2.c/blob/master/my-run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run.c


In [ ]:
#@title Save to Github

#@markdown Need to use "Save a copy in Github", from this [issue comment](https://github.com/googlecolab/colabtools/issues/2518#issuecomment-997020696)

In [ ]:
#@title Clone Project

!git clone https://github.com/karpathy/llama2.c.git
%cd llama2.c

In [ ]:
#@title Pre-request

!apt-get install tree jq

In [ ]:
#@title Explore Dataset

!python tinystories.py download

In [ ]:
#@markdown ##The dataset consists of multiple shard JSON files.

!ls -lSsh data/TinyStories_all_data

In [ ]:
print("Each json file have multiple data item:")

!cat data/TinyStories_all_data/data00.json | jq '. | length'

print("Each data item have bellow json structure:")

!cat data/TinyStories_all_data/data00.json | jq '.[0]'

## Questions

1. What's purpose for `instruction` , `summary`, `source` in data item?

In [ ]:
#@title PreTokenize
!python tinystories.py pretokenize

In [ ]:
#@title Build

!make runfast

In [ ]:
#@title Pick Your Model

#@markdown Choose model
model = "stories15M" #@param ["stories15M", "stories42M", "stories110M"]

download_url = ""

if(model == "stories15M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin"
if(model == "stories42M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin"
if(model == "stories110M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin"

print(f"download_url: {download_url}")

!wget $download_url

model_file = model + ".bin"

In [ ]:
#@title Generate Stories

# Generate args
max_token = 256 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.8 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
prompt = "One day, Lily met a Shoggoth" #@param {type:"string"}

print(f"model: {model_file}, max_token: {max_token}, temperature: {temperature}, top_p: {top_p}, prompt: {prompt}")
print(f"----------------------------\n")

cmd = f'./run {model_file} -t {temperature} -p {top_p} -n {max_token} -i "{prompt}"'
!{cmd}

In [ ]:
#@title Run Meta's Llama 2 models

#@markdown input your huggingface [access token](https://huggingface.co/settings/tokens) to download Meta's Llama 2 models.

from huggingface_hub import snapshot_download

token = "replace your huggingface access token" #@param {type:"string"}
path = snapshot_download(repo_id="meta-llama/Llama-2-7b",cache_dir="Llama-2-7b", use_auth_token=token)

!python export.py llama2_7b.bin --meta-llama $path

print("./run llama2_7b.bin\n")
!./run llama2_7b.bin